In [1]:
import cptac
import pandas as pd
import pyensembl
import scipy.stats as stats

In [2]:
ensembl = pyensembl.EnsemblRelease()

In [3]:
cptac.download(dataset="luad")
cptac.download(dataset="hnscc")
cptac.download(dataset="ovarian")
cptac.download(dataset="colon")
cptac.download(dataset="brca")
cptac.download(dataset="lscc")

True

In [4]:
luad = cptac.Luad()
brca = cptac.Brca()
ovarian = cptac.Ovarian()
colon = cptac.Colon()
lscc = cptac.Lscc()
hnscc = cptac.Hnscc()

cptac warning: The LUAD dataset is under publication embargo until July 01, 2020. CPTAC is a community resource project and data are made available rapidly after generation for community research use. The embargo allows exploring and utilizing the data, but analysis may not be published until after the embargo date. Please see https://proteomics.cancer.gov/data-portal/about/data-use-agreement or enter cptac.embargo() to open the webpage for more details. (C:\Users\chels\anaconda3\lib\site-packages\ipykernel_launcher.py, line 1)
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


version 3scc v3.2.......                    


cptac warning: The LSCC data is currently strictly reserved for CPTAC investigators. Otherwise, you are not authorized to access these data. Additionally, even after these data become publicly available, they will be subject to a publication embargo (see https://proteomics.cancer.gov/data-portal/about/data-use-agreement or enter cptac.embargo() to open the webpage for more details). (C:\Users\chels\anaconda3\lib\site-packages\ipykernel_launcher.py, line 5)


cptac warning: The HNSCC data is currently strictly reserved for CPTAC investigators. Otherwise, you are not authorized to access these data. Additionally, even after these data become publicly available, they will be subject to a publication embargo (see https://proteomics.cancer.gov/data-portal/about/data-use-agreement or enter cptac.embargo() to open the webpage for more details). (C:\Users\chels\anaconda3\lib\site-packages\ipykernel_launcher.py, line 6)


In [5]:
luad_cnv = luad.get_CNV()
hnscc_cnv = hnscc.get_CNV()
ovarian_cnv =  ovarian.get_CNV()
colon_cnv = colon.get_CNV()
brca_cnv = brca.get_CNV()
lscc_cnv = lscc.get_CNV()

In [6]:
def somatic_mutations_to_binary(df):
    new_df = df.copy()
    new_df['v'] = 1
    new_df['m'] = df['Gene'] + '_' + df['Location']
    df.reset_index()
    new_df = pd.pivot_table(new_df, values=['v'], index=['Patient_ID'], columns=['Gene'], fill_value=0)
#     print(new_df)
    if isinstance(new_df.columns, pd.MultiIndex):
        new_df.columns = new_df.columns.droplevel(0)
        new_df.columns = new_df.columns.rename("Name")
    return new_df

In [7]:
somatic_mutations_to_binary(brca.get_somatic_mutation())

Name,A1CF,A2M,A2ML1,A4GALT,AACS,AADACL2,AADACL4,AADAT,AAED1,AAGAB,...,ZSWIM5,ZSWIM8,ZUFSP,ZW10,ZWILCH,ZXDA,ZXDB,ZYG11A,ZZEF1,hsa-mir-150
Patient_ID,,,,,,,,,,,,,,,,,,,,,
CPT000814,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CPT001846,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
X01BR001,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
X01BR008,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
X01BR009,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
X21BR001,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
X21BR002,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
X21BR010,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
luad_mutations = somatic_mutations_to_binary(luad.get_somatic_mutation())
hnscc_mutations = somatic_mutations_to_binary(hnscc.get_somatic_mutation())
ovarian_mutations = somatic_mutations_to_binary(ovarian.get_somatic_mutation())
colon_mutations = colon.get_somatic_mutation_binary()
brca_mutations = somatic_mutations_to_binary(brca.get_somatic_mutation())
lscc_mutations = somatic_mutations_to_binary(lscc.get_somatic_mutation())

In [26]:
colon_mutations = somatic_mutations_to_binary(colon.get_somatic_mutation())

## Find if it has event

In [9]:
luad_has_event = pd.read_csv("luad_has_event.csv")
hnscc_has_event = pd.read_csv("hnscc_has_event.csv")
ovarian_has_event = pd.read_csv("ovarian_has_event.csv")
colon_has_event = pd.read_csv("colon_has_event.csv")
brca_has_event = pd.read_csv("brca_has_event.csv")
lscc_has_event = pd.read_csv("lscc_has_event.csv")

In [10]:
luad_mutations = luad_mutations.join(luad_has_event.set_index("Patient_ID").applymap(lambda x: int(x)))
hnscc_mutations = hnscc_mutations.join(hnscc_has_event.set_index("Patient_ID").applymap(lambda x: int(x)))
ovarian_mutations = ovarian_mutations.join(ovarian_has_event.set_index("Patient_ID").applymap(lambda x: int(x)))
colon_mutations = colon_mutations.join(colon_has_event.set_index("Patient_ID").applymap(lambda x: int(x)))
brca_mutations = brca_mutations.join(brca_has_event.set_index("Patient_ID").applymap(lambda x: int(x)))
lscc_mutations = lscc_mutations.join(lscc_has_event.set_index("Patient_ID").applymap(lambda x: int(x)))

In [27]:
colon_mutations = colon_mutations.join(colon_has_event.set_index("Patient_ID").applymap(lambda x: int(x)))

## Append Row

In [11]:
# luad_mutations['event'] = luad_mutations.index.isin(luad_has_event).astype(int)
# hnscc_mutations['event'] = hnscc_mutations.index.isin(hnscc_has_event).astype(int)
# ovarian_mutations['event'] = ovarian_mutations.index.isin(ovarian_has_event).astype(int)
# colon_mutations['event'] = colon_mutations.index.isin(colon_has_event).astype(int)
# brca_mutations['event'] = brca_mutations.index.isin(brca_has_event).astype(int)
# lscc_mutations['event'] = lscc_mutations.index.isin(lscc_has_event).astype(int)

In [12]:
# luad_has_event = pd.DataFrame({'loss_event': luad_mutations['event']}, index = luad_mutations.index)
# ovarian_has_event = pd.DataFrame({'loss_event': ovarian_mutations['event']}, index = ovarian_mutations.index)
# hnscc_has_event = pd.DataFrame({'loss_event': hnscc_mutations['event']}, index = hnscc_mutations.index)
# colon_has_event = pd.DataFrame({'loss_event': colon_mutations['event']}, index = colon_mutations.index)
# brca_has_event = pd.DataFrame({'loss_event': brca_mutations['event']}, index = brca_mutations.index)
# lscc_has_event = pd.DataFrame({'loss_event': lscc_mutations['event']}, index = lscc_mutations.index)

In [13]:
# luad_has_event.to_csv("luad_has_loss_event.csv")
# ovarian_has_event.to_csv("ovarian_has_loss_event.csv")
# hnscc_has_event.to_csv("hnscc_has_loss_event.csv")
# colon_has_event.to_csv("colon_has_loss_event.csv")
# brca_has_event.to_csv("brca_has_loss_event.csv")
# lscc_has_event.to_csv("lscc_has_loss_event.csv")

In [14]:
luad_mutations

,A1BG,A1CF,A2M,A2ML1,A4GNT,AAAS,AACS,AADACL3,AADACL4,AAK1,...,ZWILCH,ZXDA,ZXDB,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,gain_event,loss_event
Patient_ID,,,,,,,,,,,,,,,,,,,,,
C3L-00001,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
C3L-00009,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
C3L-00080,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,1
C3L-00083,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
C3L-00093,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C3N-02729,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
X11LU013,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
X11LU016,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [173]:
# luad_mutations.to_csv("luad_mutations_parm.csv")
# hnscc_mutations.to_csv("hnscc_muations_parm.csv")
# ovarian_mutations.to_csv("ovarian_mutations_parm.csv")
# colon_mutations.to_csv("colon_mutations_parm.csv")
# brca_mutations.to_csv("brca_mutations_parm.csv")
# lscc_mutations.to_csv("lscc_mutations_parm.csv")

## Run the Fishers Tests

In [18]:
def run_fishers_test(df, cancer, search_col):
    results = pd.DataFrame(columns=['Mutation', f'{cancer}_odds', f'{cancer}_pvalue'])
    cols = list(df.columns)
    cols.remove('gain_event')
    cols.remove('loss_event')
    i = 0
    for col in cols:
        table = pd.crosstab(df[col], df[search_col])
        oddsratio, pvalue = stats.fisher_exact(table)
        if pvalue < 0.005:
            print(table)
        results.loc[i] = [col, oddsratio, pvalue]
        i += 1
    return results

In [16]:
 import time

In [66]:
start = time.time()
luad_fishers_loss = run_fishers_test(luad_mutations, 'Brca', 'loss_event')
luad_fishers_loss.to_csv("fisher_results/luad_fishers_loss.csv")
print(time.time()  - start)

loss_event   0   1
EGFR              
0           56  13
1           19  21
159.870023727417


In [19]:
start = time.time()
luad_fishers_gain = run_fishers_test(luad_mutations, 'Brca', 'gain_event')
luad_fishers_gain.to_csv("fisher_results/luad_fishers_gain.csv")
print(time.time()  - start)

gain_event   0   1
AHNAK2            
0           71  21
1            6  11
gain_event   0   1
ANK2              
0           69  20
1            8  12
gain_event   0   1
ATP10B            
0           75  25
1            2   7
gain_event   0   1
BTBD11            
0           77  27
1            0   5
gain_event   0   1
CACNA1S           
0           76  25
1            1   7
gain_event   0   1
CACNA2D1          
0           75  25
1            2   7
gain_event   0   1
CCDC166           
0           76  26
1            1   6
gain_event   0   1
CUBN              
0           74  24
1            3   8
gain_event   0   1
DISP3             
0           75  25
1            2   7
gain_event   0   1
DYNC2H1           
0           75  25
1            2   7
gain_event   0   1
FLG               
0           69  18
1            8  14
gain_event   0   1
GLDC              
0           75  25
1            2   7
gain_event   0   1
GRM5              
0           73  22
1            4  10
gain_event  

In [20]:
start = time.time()
hnscc_fishers_loss = run_fishers_test(hnscc_mutations, 'Hnscc', 'loss_event')
hnscc_fishers_loss.to_csv("fisher_results/hnscc_fishers_loss.csv")
print(time.time()  - start)

loss_event   0   1
ANKRD20A1         
0           79  26
1            0   4
loss_event   0   1
ENPP2             
0           79  26
1            0   4
loss_event   0   1
IFIH1             
0           79  26
1            0   4
loss_event   0   1
RC3H2             
0           79  26
1            0   4
137.00405550003052


In [21]:
start = time.time()
hnscc_fishers_gain = run_fishers_test(hnscc_mutations, 'Hnscc', 'gain_event')
hnscc_fishers_gain.to_csv("fisher_results/hnscc_fishers_gain.csv")
print(time.time()  - start)

141.60782885551453


In [22]:
start = time.time()
ovarian_fishers_loss = run_fishers_test(ovarian_mutations, 'Ovarian', 'loss_event')
ovarian_fishers_loss.to_csv("fisher_results/ovarian_fishers_loss.csv")
print(time.time()  - start)

130.02719497680664


In [23]:
start = time.time()
ovarian_fishers_gain = run_fishers_test(ovarian_mutations, 'Ovarian', 'gain_event')
ovarian_fishers_gain.to_csv("fisher_results/ovarian_fishers_gain.csv")
print(time.time()  - start)

133.383784532547


In [28]:
start = time.time()
colon_fishers_loss = run_fishers_test(colon_mutations, 'Colon', 'loss_event')
colon_fishers_loss.to_csv("fisher_results/colon_fishers_loss.csv")
print(time.time()  - start)

loss_event   0   1
ACVR2A            
0           41  40
1           24   1
loss_event   0   1
ADAMTS5           
0           53  41
1           12   0
loss_event   0   1
ADAMTSL4          
0           53  41
1           12   0
loss_event   0   1
ANK3              
0           52  41
1           13   0
loss_event   0   1
ASH1L             
0           49  41
1           16   0
loss_event   0   1
BRAF              
0           48  40
1           17   1
loss_event   0   1
BSN               
0           49  40
1           16   1
loss_event   0   1
CAMTA1            
0           52  41
1           13   0
loss_event   0   1
CCDC168           
0           40  37
1           25   4
loss_event   0   1
CPAMD8            
0           53  41
1           12   0
loss_event   0   1
DST               
0           49  40
1           16   1
loss_event   0   1
FCGBP             
0           50  40
1           15   1
loss_event   0   1
GCN1              
0           50  40
1           15   1
loss_event  

In [29]:
start = time.time()
colon_fishers_gain = run_fishers_test(colon_mutations, 'Colon', 'gain_event')
colon_fishers_gain.to_csv("fisher_results/colon_fishers_gain.csv")
print(time.time()  - start)

gain_event   0   1
ABCA4             
0           46  46
1           13   1
gain_event   0   1
ADAMTSL4          
0           47  47
1           12   0
gain_event   0   1
ALMS1             
0           50  47
1            9   0
gain_event   0   1
ARMCX4            
0           49  47
1           10   0
gain_event   0   1
BMPR2             
0           46  46
1           13   1
gain_event   0   1
CD93              
0           50  47
1            9   0
gain_event   0   1
CELSR1            
0           44  46
1           15   1
gain_event   0   1
CHD6              
0           50  47
1            9   0
gain_event   0   1
CIC               
0           49  47
1           10   0
gain_event   0   1
COL27A1           
0           50  47
1            9   0
gain_event   0   1
DPAGT1            
0           49  47
1           10   0
gain_event   0   1
FN1               
0           50  47
1            9   0
gain_event   0   1
KIAA1429          
0           48  47
1           11   0
gain_event  

In [30]:
start = time.time()
brca_fishers_loss = run_fishers_test(brca_mutations, 'Brca', 'loss_event')
brca_fishers_loss.to_csv("fisher_results/brca_fishers_loss.csv")
print(time.time()  - start)

450.87221336364746


In [31]:
start = time.time()
brca_fishers_gain = run_fishers_test(brca_mutations, 'Brca', 'gain_event')
brca_fishers_gain.to_csv("fisher_results/brca_fishers_gain.csv")
print(time.time()  - start)

409.56441926956177


In [32]:
start = time.time()
lscc_fishers_loss = run_fishers_test(lscc_mutations, 'Lscc', 'loss_event')
lscc_fishers_loss.to_csv("fisher_results/lscc_fishers_loss.csv")
print(time.time()  - start)

loss_event   0   1
EPHB1             
0           66  36
1            0   6
loss_event   0   1
OTOF              
0           66  36
1            0   6
loss_event   0   1
RELN              
0           62  30
1            4  12
loss_event    0   1
RP5-862P8.2        
0            66  36
1             0   6
loss_event   0   1
TBC1D9B           
0           66  36
1            0   6
loss_event   0   1
VEPH1             
0           66  35
1            0   7
loss_event   0   1
XKR4              
0           66  35
1            0   7
438.6192226409912


In [33]:
start = time.time()
lscc_fishers_gain = run_fishers_test(lscc_mutations, 'Lscc', 'gain_event')
lscc_fishers_gain.to_csv("fisher_results/lscc_fishers_gain.csv")
print(time.time()  - start)

gain_event   0   1
KRT1              
0           78  25
1            0   5
gain_event   0   1
NLRP3             
0           76  22
1            2   8
gain_event   0   1
PDZD4             
0           78  24
1            0   6
gain_event   0   1
TGM3              
0           78  25
1            0   5
gain_event   0   1
WDR7              
0           78  24
1            0   6
408.13021993637085


In [61]:
luad_fishers.to_csv("luad_fishers_by_gene_loss.csv")

In [65]:
luad_fishers[luad_fishers.Luad_pvalue < 0.05].sort_values('Luad_odds')

,Mutation,Luad_odds,Luad_pvalue
6773,NTRK3,0.000000,0.016632
2683,DNAH10,0.000000,0.016366
7874,POM121L12,0.132035,0.034030
7393,PCDH15,0.160714,0.010184
9858,STK11,0.184211,0.018248
...,...,...,...
6967,OR2A12,inf,0.028504
6991,OR2H2,inf,0.028504
7361,PASD1,inf,0.028504
4216,H2BFM,inf,0.028504


In [170]:
start = time.time()
table = pd.crosstab(luad_mutations['A1BG_p.L110Q'], luad_mutations['event'])
oddsratio, pvalue = stats.fisher_exact(table)
print(time.time()  - start)

0.025036334991455078
